In [13]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [14]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [15]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

Experiment 1: Train Logistic Regression Classifier

In [16]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



Experiment 2: Train Random Forest Classifier

In [17]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       270
           1       0.91      0.70      0.79        30

    accuracy                           0.96       300
   macro avg       0.94      0.85      0.89       300
weighted avg       0.96      0.96      0.96       300



Experiment 3: Train XGBoost

In [18]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



Handle Class Imbalance using SMOTE

In [19]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

TRACK EXPERIMENTS USING MLFLOW

In [20]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [21]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    model.set_params(**params)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [22]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [24]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection_dagshub")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/12/26 22:29:34 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection_dagshub' does not exist. Creating a new experiment.
2024/12/26 22:29:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/233655457184126669/runs/aa26e96bd7074fbd99b59f5fa597406b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/233655457184126669


2024/12/26 22:29:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/233655457184126669/runs/7f0863b9e553426a9e18e54081e888e4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/233655457184126669


2024/12/26 22:29:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/233655457184126669/runs/f9ef3b468e284d999459ebfc58b21734
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/233655457184126669


2024/12/26 22:29:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/233655457184126669/runs/4adb6bb351dc4db2ba050b59f5186fd5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/233655457184126669


Register the model

In [25]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Please type RunID 4adb6bb351dc4db2ba050b59f5186fd5


Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2024/12/26 22:30:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 2


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/233655457184126669/runs/4adb6bb351dc4db2ba050b59f5186fd5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/233655457184126669


Created version '2' of model 'XGB-Smote'.


Load the model

In [26]:
model_name = 'XGB-Smote'
model_version = 2
# model_uri = f"models:/{model_name}@/challenger"
model_uri = f"models:/{model_name}/{model_version}"
try:
    # Attempt to load the model
    loaded_model = mlflow.xgboost.load_model(model_uri)
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {str(e)}")

Model loaded successfully!


In [28]:
# model_name = 'XGB-Smote'
# model_version = 1
# # model_uri = f"models:/{model_name}@/challenger"
# model_uri = f"models:/{model_name}/{model_version}"

# loaded_model = mlflow.xgboost.load_model(model_uri)
# loaded_model
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

Dagshub Setup

In [30]:
import dagshub
dagshub.init(repo_owner='ayang.laishram', repo_name='mlflow', mlflow=True)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=50a062b1-6c2a-46e6-acbc-1633ad3e6f9c&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=d7cc00f7daf8de49adefc7f199c42a8d67b980b37ea8a5c559f449efcad98860




Output()

Accessing as ayang.laishram

Initialized MLflow to track repo "ayang.laishram/mlflow"

Repository ayang.laishram/mlflow initialized!

In [32]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection_dagshub")
# mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_tracking_uri("https://dagshub.com/ayang.laishram/mlflow.mlflow")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/12/26 23:19:15 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection_dagshub' does not exist. Creating a new experiment.
2024/12/26 23:19:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: https://dagshub.com/ayang.laishram/mlflow.mlflow/#/experiments/0/runs/cfc7c6a492d44fc087610d6a656af921
🧪 View experiment at: https://dagshub.com/ayang.laishram/mlflow.mlflow/#/experiments/0


2024/12/26 23:19:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: https://dagshub.com/ayang.laishram/mlflow.mlflow/#/experiments/0/runs/66adc55d548b49c39c20134bf7d1ea16
🧪 View experiment at: https://dagshub.com/ayang.laishram/mlflow.mlflow/#/experiments/0


2024/12/26 23:19:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: https://dagshub.com/ayang.laishram/mlflow.mlflow/#/experiments/0/runs/a104b7de7268463e9f24cfec6e3f97a8
🧪 View experiment at: https://dagshub.com/ayang.laishram/mlflow.mlflow/#/experiments/0


2024/12/26 23:19:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: https://dagshub.com/ayang.laishram/mlflow.mlflow/#/experiments/0/runs/da310b5495f346688b959fffdf73640b
🧪 View experiment at: https://dagshub.com/ayang.laishram/mlflow.mlflow/#/experiments/0
